# Using PhysiCOOL with third-party libraries

The `BlackBoxModel` class can be used in combination with other libraries to perform parameter studies and optimization routines that have still not been implemented in PhysiCOOL.

Here, we describe how to couple our approach with the [psweep](https://github.com/elcorto/psweep) library, which can be used to set up and run parameter studies. The PhysiCell project used is the one described in the motility example. 

A Pandas DataFrame is created by `psweep` with the model outputs for each parameter set and some additional information (run_id, run_time...). Results are also stored in a folder called `calc`.

This approach can be used with other libraries, as users see fit.

In [9]:
from physicool. optimization import PhysiCellBlackBox
from physicool.updaters import CellUpdater, update_motility_values
from physicool.processing import get_final_y_position

import psweep as ps
import shutil

# Compiles the project and creates a black box object for it
# opt.compile_project()    
black_box = PhysiCellBlackBox()

# Define the updater we want to use (change motility data)
updater = CellUpdater(updater_function=update_motility_values,
                     config_path="config/PhysiCell_settings.xml", 
                     cell_definition_name="default")

# Assign the updater and processor to the black box
black_box.updater = updater
black_box.processor = get_final_y_position

def func(pset): 
    """
    Runs the black box model with the values selected by psweep.
    """
    shutil.rmtree("temp")
    metric = black_box.run(pset)
    return {'result': metric}

# Choose parameters
# (Creates a grid [["speed": 1.0, "bias": 0.2],
#                  ["speed": 1.0, "bias": 0.6]])
a = ps.plist('migration_bias', [0.2, 0.6])
b = ps.plist('speed', [1.0])
params = ps.pgrid(a,b)

# Get a DataFrame with all the simulation results
# psweep will run func() with each pair of the params grid
df = ps.run_local(func, params)
df.head()

,migration_bias,speed,_pset_hash,_run_id,_pset_id,_calc_dir,_time_utc,_pset_seq,_run_seq,result,_pset_runtime
0,0.2,1.0,cd3eeb3b2452a69eac62b2d2d76d9f29c1d19be4,5552a195-330e-4cce-a4ab-39694d2aa0f6,9d92eaa9-9107-4ca6-9467-b4d79289a270,calc,2022-11-14 09:44:12.222487450,0,0,"[-492.3644116448156, -280.5696112767467, -387....",3.358115
1,0.6,1.0,5a9a45aab62d3a8fed2acdc73b803e96429a24f4,5552a195-330e-4cce-a4ab-39694d2aa0f6,56d5ed87-5ea5-4733-b41f-4fab2142661f,calc,2022-11-14 09:44:15.583536148,1,0,"[-195.30425340043033, -236.24844463939684, -22...",3.126027
